# Analysis Flow

This notebook demonstrates how an Automate Flow can be used to perform analysis. 
We use a flow that combines Globus Transfer and funcX to move data and then execute an analysis code on the data.

In [1]:
from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)

import time
import json
import sys
import os

from funcx.sdk.client import FuncXClient

CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

## Create an analysis function

Use funcX to register a simple analysis function.

In [2]:
# fxc = FuncXClient()
fxc = FuncXClient(use_offprocess_checker=False)

In [3]:
import funcx
funcx.__version__

'0.2.3'

In [5]:
def file_size(pathname):
    """Return the size of a file"""
    return 1
#     import os
#     return os.path.getsize(data['pathname'])

func_uuid = fxc.register_function(file_size)
print(func_uuid)

5172f1c2-f554-4228-8c60-873a96af07d4


In [6]:
payload = {'pathname': '/etc/hostname'}
tutorial_endpoint = '4b116d3c-1703-4f8f-9f6f-39921e5864df' # Public tutorial endpoint
res = fxc.run(payload, endpoint_id=tutorial_endpoint, function_id=func_uuid)
print(res)

3e10df1d-baaf-4a98-9ea3-45ca2e8abead


In [7]:
fxc.get_result(res)

1

## Create the flow

In [8]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "funcx",
  "States": {
    "funcx": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://automate.funcx.org",
      "ActionScope": "https://auth.globus.org/scopes/b3db7e59-a6f1-4947-95c2-59d6b7a70f8c/action_all",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "function.$": "$.input.fx_id",
            "payload": {
                "pathname.$": "$.input.pathname"
            }
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 600,
      "End": True
    }
  }
}


Register the flow

In [9]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Stills process workflow")
flow_id = flow['id']
print(flow)
flow_scope = flow['globus_auth_scope']
print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/f3a0daae-2661-496f-b6d7-f7baa31c3fbb', 'administered_by': [], 'api_version': '1.0', 'created_at': '2021-05-27T17:55:41.729854+00:00', 'created_by': 'urn:globus:auth:identity:c4765424-d274-11e5-b894-cb4139f74ecf', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'funcx', 'States': {'funcx': {'ActionScope': 'https://auth.globus.org/scopes/b3db7e59-a6f1-4947-95c2-59d6b7a70f8c/action_all', 'ActionUrl': 'https://automate.funcx.org', 'Comment': 'Run a funcX function', 'End': True, 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'function.$': '$.input.fx_id', 'payload': {'pathname.$': '$.input.pathname'}}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 600}}}, 'description': '', 'flow_url': 'https://flows.globus.org/flows/f3a0daae-2661-496f-b6d7-f7baa31c3fbb', 'globus_auth_scope': 'https://auth.globus.org/scopes/f3a0daae-2661-496f-b6d7-f7baa31c3fbb/flow_f3a0daae_2661_496f_b6d7_f7baa31c3fbb_use

In [10]:
filename = 'test.txt'

flow_input = {
    "input": {
        "fx_id": func_uuid,
        "fx_ep": tutorial_endpoint,
        "pathname": '/etc/hostname'
    }
}

Run the flow and wait for it to complete

Note: This will try to transfer a file called "test.txt" from your Globus Tutorial Endpoint 1.

In [11]:
flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
# print(flow_action)
flow_action_id = flow_action['action_id']
flow_status = flow_action['status']
print(f'Flow action started with id: {flow_action_id}')
while flow_status == 'ACTIVE':
    time.sleep(5)
    flow_action = flows_client.flow_action_status(flow_id, flow_scope, flow_action_id)
    flow_status = flow_action['status']
    print(f'Flow status: {flow_status}')

Please log into Globus here:
----------------------------
https://auth.globus.org/v2/oauth2/authorize?client_id=e6c75d97-532a-4c88-b031-8584a319fa3e&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=https%3A%2F%2Fauth.globus.org%2Fscopes%2Ff3a0daae-2661-496f-b6d7-f7baa31c3fbb%2Fflow_f3a0daae_2661_496f_b6d7_f7baa31c3fbb_user&state=_default&response_type=code&code_challenge=77xk3uMMN5RoNe4dDW44XnO_r2IEgU0ypmi0zfN9umA&code_challenge_method=S256&access_type=offline&prefill_named_grant=Globus+Automate+Command+Line+Interface+on+Ryans-MacBook-Pro.local
----------------------------

Enter the resulting Authorization Code here: GtzeKIEllxRsDH1SHjRk2FD1zpDUBH
Flow action started with id: fb37e5f6-ba02-4004-b3df-46495b39a48b
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: ACTIVE
Flow status: SUCCEEDED


Check the result of the analysis function

In [12]:
flow_action['details']

{'output': {'AnalyzeResult': {'action_id': 'f38b8c38-4762-43ec-aa62-54e376881bab',
   'details': '{"223b1ce8-e33b-47f8-80c7-0cb3733a7814": {"result": 1}}',
   'display_status': 'Function Results Received',
   'state_name': 'funcx',
   'status': 'SUCCEEDED'},
  'input': {'fx_ep': '4b116d3c-1703-4f8f-9f6f-39921e5864df',
   'fx_id': '5172f1c2-f554-4228-8c60-873a96af07d4',
   'pathname': '/etc/hostname'}}}